In [1]:
!pip install keras-cv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 9.3 MB/s eta 0:00:00


In [2]:
!unzip /content/pothole.zip -d /content/dataset

Archive:  /content/pothole.zip
  inflating: /content/dataset/normal/1.jpg  
  inflating: /content/dataset/normal/10.jpg  
  inflating: /content/dataset/normal/100.jpg  
  inflating: /content/dataset/normal/101.jpg  
  inflating: /content/dataset/normal/102.jpg  
  inflating: /content/dataset/normal/103.jpg  
  inflating: /content/dataset/normal/104.jpg  
  inflating: /content/dataset/normal/105.jpg  
  inflating: /content/dataset/normal/106.jpg  
  inflating: /content/dataset/normal/107.jpg  
  inflating: /content/dataset/normal/108.jpg  
  inflating: /content/dataset/normal/109.jpg  
  inflating: /content/dataset/normal/11.jpg  
  inflating: /content/dataset/normal/110.jpg  
  inflating: /content/dataset/normal/111.jpg  
  inflating: /content/dataset/normal/112.jpg  
  inflating: /content/dataset/normal/113.jpg  
  inflating: /content/dataset/normal/114.jpg  
  inflating: /content/dataset/normal/115.jpg  
  inflating: /content/dataset/normal/116.jpg  
  inflating: /content/dataset/nor

In [6]:
!pip install fastai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 858.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.0 MB/s eta 0:00:00


In [7]:
from fastai.vision.all import *
from fastai.data.all import *

In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

# Load the pre-trained ResNet50 model
model = ResNet50(weights='imagenet')



102967424/102967424 [==============================] - 1s 0us/step


In [8]:
# Load and prepare your image(s) here
path = Path('/content/dataset')
print(path.ls())


print(get_image_files(path/"normal")[:5])

potholes = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224, min_scale=0.75)
)

[Path('/content/dataset/potholes'), Path('/content/dataset/normal')]
[Path('/content/dataset/normal/82.jpg'), Path('/content/dataset/normal/325.jpg'), Path('/content/dataset/normal/309.jpg'), Path('/content/dataset/normal/8.jpg'), Path('/content/dataset/normal/49.jpg')]


In [16]:
img = image.load_img('/content/dataset/normal/100.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

predictions = model.predict(x)

print('Predicted:', decode_predictions(predictions, top=3)[0])


1/1 [==============================] - 0s 339ms/step
Predicted: [('n06794110', 'street_sign', 0.5741842), ('n03000134', 'chainlink_fence', 0.04941808), ('n02835271', 'bicycle-built-for-two', 0.035230584)]


# Fine-tuning a pretrained backbone

Load the Pre-Trained Model

In [17]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(weights='imagenet', include_top=False)


94765736/94765736 [==============================] - 1s 0us/step


In [18]:
# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False


In [21]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [24]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


In [29]:
train_data_dir = '/content/dataset'
batch_size = 16
img_height, img_width = 224, 224

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')  # Use 'binary' for binary classification


Found 681 images belonging to 2 classes.


In [43]:
steps_per_epoch = train_generator.samples // batch_size
epochs = 1


In [44]:
validation_datagen = ImageDataGenerator(rescale=1./255)


In [45]:
validation_data_dir = '/content/dataset'
validation_batch_size = 16
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=validation_batch_size,
    class_mode='binary')  # Use 'binary' for binary classification if you have two classes


Found 681 images belonging to 2 classes.


In [46]:
validation_steps = validation_generator.samples // validation_generator.batch_size


In [47]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Add new layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # This is an example; adjust as per your last layer
x = Dense(1024, activation='relu')(x)  # A dense layer as before
new_output = Dense(1, activation='sigmoid')(x)  # New binary classification output layer

# Define the new model
new_model = Model(inputs=base_model.input, outputs=new_output)

# Compile the new model
new_model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])



In [49]:
history = new_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps)


42/42 [==============================] - ETA: 0s - loss: 0.7860 - accuracy: 0.5158

KeyboardInterrupt: 

# Training a image classifier from scratch

In [51]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [52]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Use 'softmax' if you have more than 2 classes
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Change to 'categorical_crossentropy' if you have more than 2 classes
              metrics=['accuracy'])


In [54]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for validation data

train_generator = train_datagen.flow_from_directory(
        '/content/dataset',  # This is the target directory
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        class_mode='binary')  # Use 'binary' for binary classification, 'categorical' for multi-class

validation_generator = validation_datagen.flow_from_directory(
        '/content/dataset',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')  # Use 'binary' for binary classification, 'categorical' for multi-class


Found 681 images belonging to 2 classes.
Found 681 images belonging to 2 classes.


In [55]:
history = model.fit(
      train_generator,
      steps_per_epoch=30,  # Number of images = batch_size * steps
      epochs=1,
      validation_data=validation_generator,
      validation_steps=25,  # Number of images = batch_size * steps
      verbose=2)


30/30 - 74s - loss: 0.8837 - accuracy: 0.5551 - val_loss: 0.6623 - val_accuracy: 0.5213 - 74s/epoch - 2s/step
